In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, SGD

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import load_model 

import numpy as np
import pickle
import time

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
datagen = ImageDataGenerator(rescale=1./255)


train_generator = datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/dataset/data_2categ/data_augmented/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=7
)

valid_generator = datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/dataset/data_2categ/data_augmented/validation",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=16,
    class_mode="binary",
    shuffle=True,
    seed=7
)

Found 4864 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [3]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))
initializer = 'he_normal'
model = Sequential()

model.add(Conv2D(32, (5, 5), padding='same', input_shape = (256, 256, 3), kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(256, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(128, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 64)      256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0

In [4]:
def lr_schedule(epoch):
    
    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 40: # 120
        lr *= 1e-2
    elif epoch > 20: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_simple/logs/{}".format(NAME))

#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint('best_model_test.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'],
              )


Learning rate:  0.0001


In [6]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50,
                    callbacks=[checkpoint, tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
  1/304 [..............................] - ETA: 0s - loss: 0.7763 - accuracy: 0.5625WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
304/304 [==============================] - ETA: 0s - loss: 0.6156 - accuracy: 0.6612
Epoch 00001: val_accuracy improved from -inf to 0.50000, saving model to best_model_test.h5
304/304 [==============================] - 16s 53ms/step - loss: 0.6156 - accuracy: 0.6612 - val_loss: 0.7897 - val_accuracy: 0.5000
Epoch 2/50
304/304 [==============================] - ETA: 0s - loss: 0.3472 - accuracy: 0.8491
Epoch 00002: val_accuracy improved from 0.50000 to 0.63393, saving model to best_model_test.h5
304/304 [==============================] 

Epoch 26/50
303/304 [============================>.] - ETA: 0s - loss: 0.0140 - accuracy: 0.9955
Epoch 00026: val_accuracy did not improve from 0.67857
304/304 [==============================] - 15s 50ms/step - loss: 0.0140 - accuracy: 0.9955 - val_loss: 3.1853 - val_accuracy: 0.6071
Epoch 27/50
303/304 [============================>.] - ETA: 0s - loss: 0.0229 - accuracy: 0.9922
Epoch 00027: val_accuracy did not improve from 0.67857
304/304 [==============================] - 15s 51ms/step - loss: 0.0228 - accuracy: 0.9922 - val_loss: 1.2744 - val_accuracy: 0.6339
Epoch 28/50
304/304 [==============================] - ETA: 0s - loss: 0.0172 - accuracy: 0.9949
Epoch 00028: val_accuracy improved from 0.67857 to 0.71429, saving model to best_model_test.h5
304/304 [==============================] - 16s 52ms/step - loss: 0.0172 - accuracy: 0.9949 - val_loss: 1.7041 - val_accuracy: 0.7143
Epoch 29/50
303/304 [============================>.] - ETA: 0s - loss: 0.0271 - accuracy: 0.9926
Epoch 00

In [7]:
from tensorflow.keras.models import load_model
model = load_model('best_model_test.h5')

In [8]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[1.7042665481567383, 0.7142857313156128]